# Lab 4: Creating Databases
- 3 main pathways: glycolysis, citric cycle, and pentose phosphate pathway
- Take genes from Drosophila, E. coli, and humans
- Create two tables with the following rows: 
    1. Genes
    2. Pathways
- **Genes Table:**
    - Fields: name, description, organism, and nucleotide sequence
    - Extra fields: chromosome, start and end position, strand, and translated sequence
    - Eukaryotes: nucleotide sequence should be the spliced mRNA and the coordinates should span the entire locus
- **Pathway Table:**
    - Fields: name and description
- Pick four enzymes from each pathway
- Gather information and sequences from Drosophila, E. coli, and humans for those enzymes

In [300]:
#different functions to import 
import sqlite3
from Bio import Entrez
from Bio import SeqIO
from Bio.Seq import Seq
import numpy as np
import time 
from Bio.ExPASy import Enzyme

## Pathway Table
This table was created manually. I got the Descriptions from google and it just consists of what each cycle is.

In [4]:
conn = sqlite3.connect('Lab4_database.db')
c=conn.cursor()

c.execute("""CREATE TABLE pathways (name TEXT,
                                   description TEXT);""")

c.execute("""INSERT INTO pathways (name,
                                   description)
                           VALUES ('Glycolysis',
                                   'It is the process of breaking down glucose into two molecules of pyruvate; It produces ATP')""")
#conn.commit()

c.execute("""INSERT INTO pathways (name,
                                   description)
                           VALUES ('Citric (TCA) Cycle',
                                   'It is a series of chemical reactions used by all aerobic organisms to release stored energy through the oxidation of acetyl-CoA derived from carbohydrates, fats, and proteins, into adenosine triphosphate and carbon dioxide. Also Known as KREBS cycle')""")
#conn.commit()

c.execute("""INSERT INTO pathways (name,
                                   description)
                           VALUES ('Pentose Phosphate Pathway',
                                   'It is a metabolic process parallel to glycolysis. It generates NADPH, pentoses, and ribose 5-phosphate. It does involve the oxidation of glucose, but its main purpose is anabolic rather than catabolic')""")
#conn.commit()

c.execute("SELECT * FROM pathways WHERE name= 'Citric (TCA) Cycle'")
print(c.fetchone())


('Citric (TCA) Cycle', 'It is a series of chemical reactions used by all aerobic organisms to release stored energy through the oxidation of acetyl-CoA derived from carbohydrates, fats, and proteins, into adenosine triphosphate and carbon dioxide. Also Known as KREBS cycle')


In [5]:
conn.commit()

## Genes Table
For this table, I created a for loop. This is because there were 36 rows and to insert all of them manually would take a significant amount of time. I have 3 different for loops, the difference in these for loops is that each loop is for a different organism. Each loop creates 12 rows for the respected organism. 

I got the gene IDs from the kegg server, made them into int lists and called them into my loop by indexing the list.

In [215]:
c.execute("""CREATE TABLE Genes (name TEXT,
                                 description TEXT,
                                 nucleotide_seq TEXT,
                                 organism TEXT);""")

conn.commit()

In [216]:
homosapien= [5236, 2538, 2203, 2597, 3419, 1738, 4190, 6392, 25796, 2821, 6120, 2203]

In [217]:
for i in range(11):

    handle = Entrez.efetch(db= 'nuccore', id= homosapien[i], rettype= 'gb', retmode= 'text', retmax= 1)
    record= SeqIO.read(handle, "genbank")
    n = record.name
    d= record.description 
    seq= str(record.seq)
    h= 'homosapiens'
    
    c.execute("""INSERT INTO Genes (name, 
                               description,
                               nucleotide_seq,
                               organism)
                        VALUES (?,
                                ?,
                                ?,
                                ?);
    """, (str(n), str(d), str(seq), str(h)))
    conn.commit()
    
    time.sleep(1)

In [246]:
Ecoli= [946266, 946572, 945773, 945068, 944864, 948668, 946826, 948680, 948074, 947937, 948041, 946370]

In [247]:
for i in range(11):

    handle = Entrez.efetch(db= 'nuccore', id= Ecoli[i], rettype= 'gb', retmode= 'text', retmax= 1)
    record= SeqIO.read(handle, "genbank")
    n = record.name
    d= record.description 
    seq= str(record.seq)
    h= 'e_coli'
    
    c.execute("""INSERT INTO Genes (name, 
                               description,
                               nucleotide_seq,
                               organism)
                        VALUES (?,
                                ?,
                                ?,
                                ?);
    """, (str(n), str(d), str(seq), str(h)))
    conn.commit()
    
    time.sleep(1)

In [256]:
Drosophila=[43582, 45398, 43437, 40409, 41468, 44291, 32940, 39899, 40875, 33431, 36358, 31007]

In [257]:
for i in range(11):

    handle = Entrez.efetch(db= 'nuccore', id= Drosophila[i], rettype= 'gb', retmode= 'text', retmax= 1)
    record= SeqIO.read(handle, "genbank")
    n = record.name
    d= record.description 
    seq= str(record.seq)
    h= 'drosophila'
    
    c.execute("""INSERT INTO Genes (name, 
                               description,
                               nucleotide_seq,
                               organism)
                        VALUES (?,
                                ?,
                                ?,
                                ?);
    """, (str(n), str(d), str(seq), str(h)))
    conn.commit()
    
    time.sleep(1)

In [258]:
c.close()

## Enzyme Table
This table was a bit more complex for me. My issue was extracting the EC number, each method I would use would extract the wrong ID. So I ended up grabbing it manually because I wasnt quite sure what else to do. Also, once I completed the table I realized it would have been a good idea to use the same enzyme gene relationship so I can create my associative table from their relationship. However, I realized that too late and I dont have enough time to go back and redo it.

In [321]:
conn = sqlite3.connect('Lab4_database.db')
c=conn.cursor()

c.execute("""CREATE TABLE Enzymes (name TEXT,
                                   function TEXT,
                                   ECnum INT);""")
conn.commit()

In [377]:
Entrez.email = 'rshammet@berkeley.edu'
handle= Entrez.esearch(db ='protein',
                      term = 'homo sapiens[ORGN] BRCA1',
                      sort = 'relevance',
                      idtype= 'acc')

fruitfly= ['NP_523411', 'NP_572656', 'NP_001261640', 'NP_523411','NP_650152' , 'NP_650042', 'NP_001137910', 
            'NP_001097629', 'NP_001097063', 'NP_001285908', 'NP_001247314', 'NP_001034075' ]
ec=['1.1.1.49', '3.1.1.31', '2.7.6.1', '1.1.1.49', '2.3.3.1', '4.2.1.3', '1.1.1.42', '1.2.4.2', '3.1.3.9', '5.1.3.15'
    , '4.1.2.13', '5.4.2.11' ]


for i in range(11):
    handle = Entrez.efetch(db= 'protein', id= fruitfly[i], rettype = 'gb', retmode= 'text')
    
    records= SeqIO.read(handle,"genbank")
    name= records.description
    function= records.features
    time.sleep(1)
    
    c.execute("""INSERT INTO Enzymes (name, 
                                      function,
                                      ECnum)
                              VALUES (?,
                                      ?,
                                      ?);
    """, (str(name), str(function[1]), str(ec[i])))
    conn.commit()
    
    time.sleep(1)

In [387]:
human= ['NP_002624', 'NP_000153', 'NP_001121089', 'NP_001280014', 'NP_004068','NP_001265281', 'NP_001269315', 
        'NP_002532', 'NP_004674', 'NP_001035810', 'NP_001055', 'NP_954699']
ec=['5.4.2.2','2.7.1.2', '4.1.2.13', '5.4.2.4', '2.3.3.1', '4.2.1.3', '1.1.1.42', '1.2.4.2', '3.1.1.17',
    '1.1.1.49', '2.2.1.1', '5.1.3.1']


for i in range(11):
    handle = Entrez.efetch(db= 'protein', id= human[i], rettype = 'gb', retmode= 'text')
    
    records= SeqIO.read(handle,"genbank")
    name= records.description
    function= records.features
    time.sleep(1)
    
    c.execute("""INSERT INTO Enzymes (name, 
                                      function,
                                      ECnum)
                              VALUES (?,
                                      ?,
                                      ?);
    """, (str(name), str(function[1]), str(ec[i])))
    conn.commit()
    
    time.sleep(1)

In [388]:
ecoli= ['NP_414666', 'NP_417894', 'NP_416960', 'NP_418449','NP_415248', 'NP_415292', 'NP_415654', 'NP_415254',
        'NP_415522', 'NP_416600', 'NP_417862', 'NP_414656']
ec=['1.1.5.2', '2.7.1.12', '2.2.1.1', '5.3.1.9','2.3.3.1', '4.2.1.3', '1.1.1.42', '1.2.4.2','3.1.3.10', '4.1.2.13', 
    '4.1.1.49', '1.2.4.1']


for i in range(11):
    handle = Entrez.efetch(db= 'protein', id= ecoli[i], rettype = 'gb', retmode= 'text')
    
    records= SeqIO.read(handle,"genbank")
    name= records.description
    function= records.features
    time.sleep(1)
    
    c.execute("""INSERT INTO Enzymes (name, 
                                      function,
                                      ECnum)
                              VALUES (?,
                                      ?,
                                      ?);
    """, (str(name), str(function[1]), str(ec[i])))
    conn.commit()
    
    time.sleep(1)

In [413]:
c.close()

## Associative Table
This table compares the gene name, with the enzyme name, and gives the EC number for the pair

In [414]:
conn = sqlite3.connect('Lab4_database.db')
c=conn.cursor()

c.execute("""CREATE TABLE Assoc (name_gene TEXT,
                                 name_enzyme TEXT,
                                 ECnum INT);""")
conn.commit()

In [446]:
c.execute("""SELECT name FROM Genes""")
gname= c.fetchall()
    
c.execute("""SELECT name FROM Enzymes""")
ename= c.fetchall()
    
c.execute("""SELECT ECnum FROM Enzymes""")
ecnum= c.fetchall()

In [ ]:
for i in range(33):    
    c.execute("""INSERT INTO Assoc (name_gene,
                                    name_enzyme,
                                    ECnum)
                            VALUES (?,
                                    ?,
                                    ?);
    """, (str(gname[i]), str(ename[i], str(ecnum[i])))
    conn.commit()
    time.sleep(1)
    